In [2]:
import numpy as np

import pandas as pd

In [3]:
df = pd.read_csv('../Data/Model_Data.csv')

In [6]:
from imblearn.combine import SMOTEENN
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, classification_report
from imblearn.pipeline import Pipeline
from xgboost import XGBClassifier

X = df.drop(columns='label')
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score),
}


estimator = Pipeline([
    ('scaler', StandardScaler()), 
    ('classification', XGBClassifier(eval_metric='auc', use_label_encoder=False)) 
])

param_grid = {
    'classification__max_depth': [1,3,5, 7, 9, 15], 
    'classification__n_estimators': [200, 300, 500],  
    'classification__learning_rate': [0.001, 0.005, 0.01], 
    'classification__colsample_bytree': [0.7, 0.8, 1.0],
    'classification__subsample': [0.8, 1.0],
    'classification__min_child_weight': [1, 2, 3], 
    'classification__scale_pos_weight': [5, 10, 20, 30, 50, 100]  
}

random_search = RandomizedSearchCV(
    estimator=estimator,
    param_distributions=param_grid,
    cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=42),
    scoring=scoring,
    refit='f1',  
    n_iter=50,  
    verbose=1
)


random_search.fit(X_train, y_train)

best_model = random_search.best_estimator_

y_pred = best_model.predict(X_test)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

joblib.dump(best_model, 'best_xgb_model_tuned.pkl')


Fitting 3 folds for each of 50 candidates, totalling 150 fits


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [04:49:07] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [04:49:08] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [04:49:08] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [04:49:09] WARNING: /Users/runner/work/xgboost/xgboost/src


Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.77      0.83      1731
           1       0.50      0.75      0.60       553

    accuracy                           0.76      2284
   macro avg       0.70      0.76      0.72      2284
weighted avg       0.81      0.76      0.77      2284



NameError: name 'joblib' is not defined